In [4]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

# --- CONFIGURATION ---
BASE_URL = "http://127.0.0.1:8000"
LOGIN_URL = "http://127.0.0.1:8000/login/"
CAMPAIGN_URL = "http://127.0.0.1:8000/campaign/11/"   # adjust ID as needed
USERNAME = "investor_user1234"                  # <-- replace with valid investor username
PASSWORD = "TestPassword123"              # <-- replace with valid password

# --- SETUP DRIVER ---
driver = webdriver.Chrome()  # or webdriver.Firefox(), etc.
driver.maximize_window()
wait = WebDriverWait(driver, 10)

try:
    # --- Step 1: Login ---
    print("🔐 Navigating to login page...")
    driver.get(LOGIN_URL)

    # Wait for username and password fields
    username_input = wait.until(EC.visibility_of_element_located((By.NAME, "username")))
    password_input = wait.until(EC.visibility_of_element_located((By.NAME, "password")))

    # Fill in credentials
    username_input.send_keys(USERNAME)
    password_input.send_keys(PASSWORD)
    password_input.submit()
    print(f"✅ Logged in as {USERNAME}")
    time.sleep(2)


    # --- Step 2: Go to Campaign Details page ---
    driver.get("http://127.0.0.1:8000/campaign/11/")
    print("✅ Opened campaign details page")
    time.sleep(1)


    # --- Step 3: Wait for campaign info to load ---
    campaign_card = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".campaign-card")))
    print("✅ Campaign card loaded")
    time.sleep(1)


    # --- Step 4: Verify campaign title & business name ---
    title = driver.find_element(By.TAG_NAME, "h2").text
    assert len(title) > 0, "Campaign title missing"
    print(f"🟢 Campaign title: {title}")

    business = driver.find_element(By.XPATH, "//p[contains(., 'Business:')]").text
    print(f"🟢 Business info found: {business}")

    # --- Step 5: Progress bar and funding info ---
    progress_bar = driver.find_element(By.CSS_SELECTOR, ".progress-bar")
    percent = progress_bar.get_attribute("data-target")
    print(f"📊 Progress: {percent}%")

    status_text = driver.find_element(By.XPATH, "//li[contains(., 'Status:')]").text
    print(f"💰 {status_text}")

    # --- Step 6: Click “Invest in this Campaign” button ---
    try:
        invest_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Invest in this Campaign')]"))
        )
        print("🟢 Found 'Invest in this Campaign' button — clicking...")
        invest_button.click()

        # Verify redirect to investment form
        wait.until(EC.url_contains("/invest"))
        print("🎉 Redirected to Investment Page successfully!")
    except TimeoutException:
        print("⚠️ Invest button not found — possibly campaign is already funded.")

    # --- Step 7: Verify Investment Page Elements ---
    try:
        amount_field = wait.until(EC.visibility_of_element_located((By.ID, "amount")))
        print("✅ Investment form loaded (amount field visible)")
    except TimeoutException:
        print("⚠️ Investment page did not load as expected")

    # --- Step 8: Navigate back to campaign details ---
    driver.back()
    print("🔙 Returned to campaign details page")

    # --- Step 9: Check for Updates section ---
    updates_section = wait.until(EC.visibility_of_element_located((By.XPATH, "//h4[contains(text(), 'Campaign Updates')]")))
    print("📰 Campaign Updates section found")

    try:
        no_updates = driver.find_element(By.XPATH, "//div[contains(text(), 'No updates posted yet.')]")
        print("ℹ️ No updates posted yet — verified")
    except:
        updates = driver.find_elements(By.CSS_SELECTOR, ".border-bottom.pb-2")
        print(f"🟢 Found {len(updates)} campaign update(s)")

    print("✅ Investor flow tested successfully!")

except AssertionError as e:
    print(f"❌ Assertion failed: {e}")
except TimeoutException as e:
    print(f"❌ Timeout while waiting for element: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
finally:
    driver.quit()
    print("🚪 Browser closed")


🔐 Navigating to login page...
✅ Logged in as investor_user1234
✅ Opened campaign details page
✅ Campaign card loaded
🟢 Campaign title: servo motor
🟢 Business info found: Business: micro processor
📊 Progress: 0.00%
💰 Status: Open
🟢 Found 'Invest in this Campaign' button — clicking...
🎉 Redirected to Investment Page successfully!
✅ Investment form loaded (amount field visible)
🔙 Returned to campaign details page
📰 Campaign Updates section found
ℹ️ No updates posted yet — verified
✅ Investor flow tested successfully!
🚪 Browser closed
